# Version 1: Stock Pricing Data

Currently there are 11 Stock Market Sectors:
- Energy: Companies that do business in the oil and natural gas industry, including exploration and production.
- Materials: Companies that provide various goods for use in manufacturing and other applications, makers of chemicals, construction materials, containers, etc.
- Industrials: Companies over a wide range of businesss such as Transportation (airlines, rr, etc.), Aerospace, Defense, Construction, Engineering.
- Utilities: Companies that provide electrical power, natural gas transmission and distribution, renewable energy.
- Healthcare: Made up of two primary components: Companies that develop pharma and treatments, Companies that create and provide healthcare equipment and services.
- Financials: Companies that handle money: Banks, Insurance Companies, Brokerage Houses, etc.
- Consumer Discretionary: Companies that sell higher priced items like automobiles, luxury goods, leisure products
- Consumer Staples: Food, beverage, and tobacco companies
- Information Technology: Computer Hardware, Software, Cybersecurity, etc.
- Communication Services: Telecommmunication Services, Media, and Entertainment
- Real Estate: Made up of two primary components: Companies responsible for developing new real estate projects, REITS (real estate investment trusts).

Version 1 of this project will include only Stock information, but as such will include all 11 sectors.

Using the initial version of this project, we will use the nasdaq and nyse csvs to pull all the various individual ticker values to pull the required information

In [112]:
import os
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf

import datetime
import get_methods

In [62]:
# due to file size, set low_memory to False to alter load order
nasdaq = pd.read_csv("./data/nasdaq_csv.csv",index_col=0, low_memory=False)
nyse = pd.read_csv("./data/nyse_csv.csv",index_col=0, low_memory=False)

In [63]:
nasdaq_tickers = nasdaq['ticker'].unique().tolist()

In [64]:
nyse_tickers = nyse['ticker'].unique().tolist()

In [65]:
len(nyse_tickers), len(nasdaq_tickers)

(1145, 1564)

We have ~1k NYSE stock tickers and ~1.5K Nasdaq stock tickers to work with, let's jump to pandas-datareaders and start crafting a function that will pull the necessary ticker information and create a dataframe for us to host on AWS RDS

---
## Extract Stock Pricing Information by Ticker using pandas-datareader and yahoo! finance

To begin let's try 10 years worth of stock pricing data 2013 to 2023:

In [94]:
start = datetime.datetime(2013,7,19)
end = datetime.datetime(2023,7,19)

Let's test 1 stock ticker to understand the response and how we can:
- make a re-usable function to capture ticker prices
- create a postgresql database to store this information

In [113]:
yfin = yf.pdr_override()

In [114]:
test = web.get_data_yahoo('GOOG')

[*********************100%***********************]  1 of 1 completed


In [115]:
test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4764 entries, 2004-08-19 to 2023-07-24
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       4764 non-null   float64
 1   High       4764 non-null   float64
 2   Low        4764 non-null   float64
 3   Close      4764 non-null   float64
 4   Adj Close  4764 non-null   float64
 5   Volume     4764 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 260.5 KB


In [100]:
test.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-07-17,126.059998,127.279999,124.500000,125.059998,125.059998,20675300
2023-07-18,124.904999,124.989998,123.300003,124.080002,124.080002,21071200
2023-07-19,124.790001,125.470001,122.470001,122.779999,122.779999,22313800
2023-07-20,122.120003,124.699997,118.684998,119.529999,119.529999,27541700
2023-07-21,120.870003,121.300003,119.080002,119.720001,119.720001,22020584


Considering the ticker is not present in the table, and that we will be pulling considerable amounts of data for ~2500 tickers, it is probably best to store this information within a PostgreSQL DB and host on AWS to save local memory and performance.

To help performance but also be able to keep records of multiple stocks and their asscoiated time series data we will set up the database with two initial Tables:
- Ticker Table
    - Columns: 'ticker_id' (Primary key), 'ticker_symbol'
    - This table will store information about each stock ticker
    - Index on 'ticker_symbol' to speed up searches on specific stocks
- Price Table
    - Columns: 'price_id' (Primary key), 'ticker_id' (Foreign key referencing the Ticker table), 'date', 'open', 'high', 'low', 'close', 'adj_close', 'volume'
    - This table will store the stock price data for each ticker
    - Potentially create an index on both the 'ticker_id' and 'date' column for efficient time-based queries 
    
Other considerations:
- For 'price_id' and 'ticker_id' we will need to ensure uniqueness to prevent collisions between records, to start we will use the SERIAL datatype for the different ids which should be managed by PostgreSQL

### Next Steps:

1. Create our PostgreSQL Database
2. Create the Ticker Table
3. Create the Price Table
4. Load Ticker Data
5. Load Price Data

Steps 1: Create PostgreSQL Database with psycopg2

In [2]:
import psycopg2

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import getpass

Password Protection

In [3]:
postgres_password = getpass.getpass("Enter PostgreSQL password: ")

Enter PostgreSQL password:  ········


establish connection with local postgresql server

In [40]:
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    password=postgres_password
)

In [24]:
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

create cursor to execute SQL commands

In [25]:
cursor = conn.cursor()

create database

In [26]:
db_name = "stock_info"

In [27]:
sqlCreateDatabase = "create database" + db_name + ";"

execute command

In [28]:
cursor.execute(create_db_query)

In [43]:
conn.close()

Steps 2-3: Create the ticker and price tables

In [42]:
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    database="stock_info",
    password=postgres_password
)

In [31]:
cursor = conn.cursor()

SQL command to create Ticker Table including:
- ticker_id as SERIAL PRIMARY KEY
- ticker_symbol as VARCHAR(10) NOT NULL

In [32]:
create_ticker_table_query = """
CREATE TABLE ticker (
    ticker_id SERIAL PRIMARY KEY,
    ticker_symbol VARCHAR(10) NOT NULL
);
"""

execute table creation

In [33]:
cursor.execute(create_ticker_table_query)

SQL command to create price Table including:
- price_id as SERIAL PRIMARY KEY
- ticker_id as foreign key
- date DATE
- open NUMERIC
- high NUMERIC
- low NUMERIC
- close NUMERIC
- adj_close NUMERIC
- volume BIGINT

In [34]:
create_price_table_query = """
CREATE TABLE price (
    price_id SERIAL PRIMARY KEY,
    ticker_id INT REFERENCES ticker(ticker_id),
    date DATE,
    open NUMERIC,
    high NUMERIC,
    low NUMERIC,
    close NUMERIC,
    adj_close NUMERIC,
    volume BIGINT
);
"""

In [35]:
cursor.execute(create_price_table_query)

In [36]:
conn.commit()

In [37]:
conn.close()

Database and inner tables successfully created!

Steps 2a: add a column for exchange to denote whether the stock is on NASDAQ or NYSE

In [44]:
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    database="stock_info",
    password=postgres_password
)

In [45]:
cursor = conn.cursor()

In [46]:
alter_table_query = "ALTER TABLE ticker ADD COLUMN exchange VARCHAR(10);"

In [47]:
cursor.execute(alter_table_query)

In [49]:
conn.commit()

In [50]:
conn.close()

Steps 4-5: Load Ticker and Price Data into respective tables

Now that we have the database and associated tables set up, let's take ticker symbols and transform them into dictionaries, combine them, and then upload them to the Ticker Table in our stock_info database

In [66]:
nasdaq_tickers = [{"ticker_symbol": ticker, "exchange": "NASDAQ"} for ticker in nasdaq_tickers]

In [67]:
nyse_tickers = [{"ticker_symbol": ticker, "exchange": "NYSE"} for ticker in nyse_tickers]

In [68]:
all_tickers = nasdaq_tickers + nyse_tickers

In [69]:
len(all_tickers)

2709

In [83]:
all_tickers[:5]

[{'ticker_symbol': 'CSCO', 'exchange': 'NASDAQ'},
 {'ticker_symbol': 'BIOS', 'exchange': 'NASDAQ'},
 {'ticker_symbol': 'CSBK', 'exchange': 'NASDAQ'},
 {'ticker_symbol': 'SBGI', 'exchange': 'NASDAQ'},
 {'ticker_symbol': 'UFCS', 'exchange': 'NASDAQ'}]

In [71]:
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    database="stock_info",
    password=postgres_password
)

In [72]:
cursor = conn.cursor()

Create query to be utilized in a for-loop for inserting ticker information from our all_tickers list

In [73]:
insert_query = """ 
INSERT INTO ticker (ticker_symbol, exchange)
VALUES (%(ticker_symbol)s, %(exchange)s);
"""

Loop through all_tickers and insert data into the 'ticker' table

In [74]:
for ticker_pair in all_tickers:
    cursor.execute(insert_query, ticker_pair)

In [75]:
conn.commit()

In [76]:
conn.close()

Let's check our work to see if this uploaded successfully!

In [134]:
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    database="stock_info",
    password=postgres_password
)

In [135]:
cursor = conn.cursor()

In [136]:
select_query = """
SELECT *
FROM ticker
LIMIT 5;
"""

In [137]:
cursor.execute(select_query)

In [138]:
head = cursor.fetchall()

In [139]:
print("Ticker ID  |  Ticker Symbol  | Exchange")
print("----------------------------------------")
for row in head:
    print(f"{row[0]:^10} | {row[1]:^14}  | {row[2]:^10}")

Ticker ID  |  Ticker Symbol  | Exchange
----------------------------------------
    1      |      CSCO       |   NASDAQ  
    2      |      BIOS       |   NASDAQ  
    3      |      CSBK       |   NASDAQ  
    4      |      SBGI       |   NASDAQ  
    5      |      UFCS       |   NASDAQ  


Success!

Now that we have successfully inserted our ticker info, let's use that data to pull all the available pricing information from yahoo finance into our price table

In [140]:
select_tick_query = """
SELECT ticker_id, ticker_symbol
FROM ticker;
"""

In [141]:
cursor.execute(select_tick_query)

Although we already have the ticker symbols in a list named 'all_tickers', I want to get in the habit of querying via SQL and using psycopg2 outputs

In [142]:
ticker_data = [{"ticker_id": row[0], "ticker_symbol": row[1]} for row in cursor.fetchall()]

In [143]:
ticker_data[0]

{'ticker_id': 1, 'ticker_symbol': 'CSCO'}

In [144]:
for info in ticker_data:
    ticker_symbol = info['ticker_symbol']
    ticker_id = info['ticker_id']
    # fetch the time series price data using pdr and yfin
    df = web.get_data_yahoo(ticker_symbol)
    # reset the index so the 'Date' column is a regular column
    df.reset_index(inplace=True)
    # iterate through the ticker dataframe and set values
    for index, row in df.iterrows():
        date = row['Date']
        open_price = row['Open']
        high_price = row['High']
        low_price = row['Low']
        close_price = row['Close']
        adj_close_price = row['Adj Close']
        volume = row['Volume']
        # set SQL query to insert the price data into the price table
        insert_query = """ 
        INSERT INTO price (ticker_id, date, open, high, low, close, adj_close, volume)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """
        # execute the query
        cursor.execute(insert_query, (ticker_id, date, open_price, high_price, low_price, close_price, adj_close_price, volume))
        
# commit the changes and close the connection
conn.commit()
conn.close()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


1 Failed download:
['WSTG']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******


1 Failed download:
['AVEO']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


1 Failed download:
['NEE-PP']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%***********************]  1 of 1 completed


1 Failed download:
['TA']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%*******

Let's check if our query worked by doing a similar 5 row limit search

In [98]:
conn = psycopg2.connect(
    host="localhost",
    user="postgres",
    database="stock_info",
    password=postgres_password
)

In [99]:
cursor = conn.cursor()

In [6]:
select_price_query = """
SELECT *
FROM price
LIMIT 5;
"""

In [7]:
cursor.execute(select_price_query)

In [8]:
price_head = cursor.fetchall()

In [29]:
print("Price ID  | Ticker ID  |  Date      | open  |  high   | low    |  close  |  adj_close  |   volume  |")
print("----------------------------------------------------------------------------------------------------")
for row in price_head:
    print(f"{row[0]:^9} | {row[1]:^9}  | {row[2]} | {row[3]:^5} | {round(row[4],4)}  | {round(row[5],4)} | {round(row[6],4)}  | {round(row[7],4):^11} | {row[8]}")

Price ID  | Ticker ID  |  Date      | open  |  high   | low    |  close  |  adj_close  |   volume  |
----------------------------------------------------------------------------------------------------
    1     |     1      | 1990-02-16 |  0.0  | 0.0799  | 0.0738 | 0.0773  |   0.0536    | 940636800
    2     |     1      | 1990-02-20 |  0.0  | 0.0799  | 0.0747 | 0.0799  |   0.0554    | 151862400
    3     |     1      | 1990-02-21 |  0.0  | 0.0790  | 0.0755 | 0.0781  |   0.0542    | 70531200
    4     |     1      | 1990-02-22 |  0.0  | 0.0816  | 0.0790 | 0.0790  |   0.0548    | 45216000
    5     |     1      | 1990-02-23 |  0.0  | 0.0799  | 0.0781 | 0.0786  |   0.0545    | 44697600


Looking good so far except for `open` being zero, let's do a quick check to see if there are any rows with open populated with a value other than zero

In [48]:
open_percentage_query = """ 
SELECT ROUND(((SELECT COUNT(open) FROM price WHERE open > 0.0) / COUNT(*)::numeric) * 100, 4) AS percentage
FROM price;
"""

In [49]:
cursor.execute(open_percentage_query)

In [50]:
open_percentage = cursor.fetchall()

In [51]:
open_percentage

[(Decimal('95.6605'),)]

Only ~5% of records in the total table have open with a value of 0.0. Given the total number of records within the database, this isn't as bad a problem as I initially anticipated. Also the number of reasons as to why this data could be 0.0 are acceptable:
- On certain dates trading for particular stock may have been halted or suspeended leading to an absence of opening prices
- Some dates could be the IPO date thus having no historical data before that entry
- Certain corporate actions such as stock splits, mergers, or acquisitions can impact historical price data
- Data Source/Data Processing errors can lead to missing or incorrect price data

Out of curiosity, we are going to amend the original head query to merge with the ticker table to see which ticker is reporting 0.0 for those dates and we can investigate

In [66]:
select_price_query_join = """
SELECT price.date, price.open, ticker.ticker_symbol
FROM price
    INNER JOIN ticker ON ticker.ticker_id = price.ticker_id
LIMIT 5;
"""

In [67]:
cursor.execute(select_price_query_join)

In [68]:
open_head_ticker = cursor.fetchall()

In [92]:
print("Date       | Open  |  Symbol |")
print("------------------------------")
for row in open_head_ticker:
    print(f"{row[0]} | {row[1]:^4}  | {row[2]:^7} |")

Date       | Open  |  Symbol |
------------------------------
1990-02-16 | 0.0   |  CSCO   |
1990-02-20 | 0.0   |  CSCO   |
1990-02-21 | 0.0   |  CSCO   |
1990-02-22 | 0.0   |  CSCO   |
1990-02-23 | 0.0   |  CSCO   |


Cisco System Inc. (CSCO) IPO'd on February 16th, 1990 which explains the first record of open = 0.0. Let's do one more query on CSCO just to see if open ever equals anything above 0.0

In [104]:
select_CSCO_open_greater = """
SELECT price.date, price.open, ticker.ticker_symbol
FROM price
    INNER JOIN ticker ON ticker.ticker_id = price.ticker_id
WHERE open > 0.0 AND ticker.ticker_symbol = 'CSCO'
ORDER BY price.date ASC
LIMIT 5;
"""

In [105]:
cursor.execute(select_CSCO_open_greater)

In [106]:
CSCO_open_greater = cursor.fetchall()

In [110]:
print("Date       | Open    |  Symbol |")
print("--------------------------------")
for row in CSCO_open_greater:
    print(f"{row[0]} | {round(row[1],4):^4}  | {row[2]:^7} |")

Date       | Open    |  Symbol |
--------------------------------
1990-03-26 | 0.0825  |  CSCO   |
1990-03-27 | 0.0833  |  CSCO   |
1990-03-28 | 0.0859  |  CSCO   |
1990-03-29 | 0.0851  |  CSCO   |
1990-03-30 | 0.0833  |  CSCO   |


In [111]:
conn.close()

The first record of CSCO where open does not equal zero is near the end of march in the same year.

Now that we have successfully loaded the available data from the tickers we initially had access to, and the data seems to be complete, let's move onto Data Cleaning and Data Exploration in the next notebook.

Once we have a working local copy of our application/dashboard, we will continue with the below steps:

6. Create an Amazon RDS Instance
7. Connect to the Amazon RDS Instance
8. Import the schema
9. Load the Data
10. Verify the data